%pip install langdetect


%pip install SpeechRecognition pyttsx3


%pip install gTTS


%pip install azure-cognitiveservices-speech


%pip install "dill==0.3.1.1" "grpcio==1.65.0" "protobuf==3.20.3"



%pip install google-cloud-texttospeech


%pip install google-cloud-speech


In [1]:
import os
from dotenv import load_dotenv
from youtube_transcript_api import YouTubeTranscriptApi
import pandas as pd
import re
import openai
import chromadb
import time
import whisper
import gradio as gr
from langchain.agents import initialize_agent, AgentType
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.tools import Tool
from langchain.memory import ConversationBufferMemory
from langdetect import detect  # New library for language detection
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [2]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")


# Load Whisper model for audio transcription
whisper_model = whisper.load_model("base")


# Define video IDs
video_ids = ['hZytp1sIZAw', 'qP1Fw2EpwqE', 'JuhBs44odO0']

c:\Users\fmrol\anaconda3\envs\newenv\lib\site-packages\whisper\__init__.py:150: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(fp, map_location=device

# Model Development 

Helper Funtions for Multilanguage QA

In [3]:
prompts = {
    "en": "You are Princess Zelda from 'The Legend of Zelda' series. Answer based on the information retrieved from the database and stay in character. "
          "Use a regal tone, and answer as if you were speaking directly to someone in the realm of Hyrule.",
    "es": "Eres la Princesa Zelda de la serie 'The Legend of Zelda'. Responde en base a la información obtenida de la base de datos y mantente en tu personaje. "
          "Usa un tono regio y responde como si estuvieras hablando directamente a alguien en el reino de Hyrule.",
    "fr": "Vous êtes la Princesse Zelda de la série 'The Legend of Zelda'. Répondez en vous basant sur les informations extraites de la base de données et restez dans le personnage. "
          "Utilisez un ton royal et répondez comme si vous parliez directement à quelqu'un dans le royaume d'Hyrule.",
    "de": "Du bist Prinzessin Zelda aus der Serie 'The Legend of Zelda'. Antworte basierend auf den Informationen aus der Datenbank und bleibe in deiner Rolle. "
          "Verwende einen königlichen Ton und antworte, als würdest du direkt mit jemandem im Reich von Hyrule sprechen.",
    "pt": "Você é a Princesa Zelda da série 'The Legend of Zelda'. Responda com base nas informações obtidas do banco de dados e mantenha-se no personagem. "
          "Use um tom régio e responda como se estivesse falando diretamente com alguém no reino de Hyrule.",
    "it": "Sei la Principessa Zelda della serie 'The Legend of Zelda'. Rispondi basandoti sulle informazioni recuperate dal database e rimani nel personaggio. "
          "Usa un tono regale e rispondi come se stessi parlando direttamente a qualcuno nel regno di Hyrule."
}

def detect_language(text):
    try:
        return detect(text)
    except:
        return "en"  # Defaults to English if detection fails

def get_prompt(language_code):
    return prompts.get(language_code, prompts["en"])


Transcript Collection and Preprocessing: 

In [4]:
def get_transcript(video_id):
    try:
        transcript = YouTubeTranscriptApi.get_transcript(video_id, languages=['en-GB'])
        transcript_text = " ".join([item['text'] for item in transcript])
        return re.sub(r'\s+', ' ', transcript_text).strip()
    except Exception as e:
        print(f"Error retrieving transcript for video {video_id}: {e}")
        return None

transcripts = {video_id: get_transcript(video_id) for video_id in video_ids}
transcripts_df = pd.DataFrame(list(transcripts.items()), columns=['Video ID', 'Transcript'])
transcripts_df

,Video ID,Transcript
0,hZytp1sIZAw,Breath of the Wild was a game about loneliness...
1,qP1Fw2EpwqE,Ganondorf’s resurrection changed Hyrule foreve...
2,JuhBs44odO0,Ever since Link and Zelda had ventured below t...


In [5]:
# Save all transcripts to a single .txt file
with open("transcripts.txt", "w", encoding="utf-8") as file:
    for index, row in transcripts_df.iterrows():
        file.write(f"Video ID: {row['Video ID']}\n")
        file.write("Transcript:\n")
        file.write(row['Transcript'] + "\n")
        file.write("-" * 80 + "\n\n")  # Separator between transcripts

print("All transcripts have been saved to transcripts.txt.")


All transcripts have been saved to transcripts.txt.


Chunking and Embedding Storage: 

In [6]:
chroma_client = chromadb.Client()
collection = chroma_client.get_or_create_collection(name="totk_transcripts")

def split_text(text, max_tokens=4000):
    words = text.split()
    chunks = []
    current_chunk = []
    current_tokens = 0

    for word in words:
        current_tokens += 1
        current_chunk.append(word)

        if current_tokens >= max_tokens:
            chunks.append(" ".join(current_chunk))
            current_chunk = []
            current_tokens = 0

    if current_chunk:
        chunks.append(" ".join(current_chunk))

    return chunks

In [7]:
def store_transcript_embeddings(video_id, transcript_text):
    # Split transcript into chunks
    text_chunks = split_text(transcript_text)
    print("Text chunks:", text_chunks)  # Debug to check chunks

    for i, chunk in enumerate(text_chunks):
        # Generate embedding for each chunk
        embedding = openai.Embedding.create(input=[chunk], model="text-embedding-ada-002")['data'][0]['embedding']
        
        # Define a unique ID
        chunk_id = f"{video_id}_chunk_{i}"
        
        try:
            collection.add(
                ids=[chunk_id],
                embeddings=[embedding],
                metadatas=[{'video_id': video_id, 'chunk_index': i, 'text': chunk}]
            )
            print(f"Stored chunk {chunk_id} successfully.")
        except Exception as e:
            print(f"Error storing chunk {chunk_id}: {e}")

for index, row in transcripts_df.iterrows():
    store_transcript_embeddings(row['Video ID'], row['Transcript'])


all_items = collection.get(include=['metadatas', 'embeddings'])
for item_metadata in all_items['metadatas']:
    print("Metadata:", item_metadata)


Text chunks: ['Breath of the Wild was a game about loneliness, an amnesiac Hero uncovering the truth of his Kingdom’s collapse, learning to survive and overcome the hostile wilderness left after the Great Calamity. Its direct sequel, Tears of the Kingdom, is instead a game about connections, about bringing Hyrule’s next generation together to fight against an ancient, yet familiar evil. Both games are set in the Kingdom of Hyrule - a vast land steeped in history and legend. The story of this Kingdom stretches back thousands upon thousands of years, though many details from its histories have been eroded or distorted by the passage of time, or even lost entirely. Breath of the Wild explains that Hyrule’s history is inextricably linked with the story of the Calamity Ganon - a primal evil that has appeared time and time again. An ancient legend tells of a war 10,000 years ago, where an emergence of the Calamity had been repelled by a Hero, a Princess, and the power of the Sheikah tribe. T

Checking cosine similarity scores

In [8]:
#Checking Cosine Similarity

query_embedding = openai.Embedding.create(input=["Who is Princess Zelda?"], model="text-embedding-ada-002")['data'][0]['embedding']

# Retrieve all stored embeddings from ChromaDB
all_embeddings = collection.get(include=['embeddings'])['embeddings']

# Compute similarity
similarities = cosine_similarity([query_embedding], all_embeddings)

# Get the top 5 most similar embeddings
top_matches = np.argsort(similarities[0])[-3:][::-1]  # Indices of top matches
print("Top similarity scores:", similarities[0][top_matches])

Top similarity scores: [0.85408565 0.85135783 0.8512961 ]


# Question-Answering Model and Retrieval System Setup


Query Processing and Semantic Search:


In [9]:
#QA Model and Retrieval System Setup
def truncate_text(text, max_tokens=3000):
    words = text.split()
    if len(words) > max_tokens:
        return " ".join(words[:max_tokens])
    return text

def multi_query_processing(user_query):
    related_queries = [
        user_query,
        f"Detect the language of {user_query}",
        f"Background on {user_query}",
        f"Historical context of {user_query}",
        f"Role of {user_query} in Hyrule",
        f"Significance of {user_query}"
        f"Respond in the same language as {user_query}"
    ]
    all_retrieved_texts = []

    for sub_query in related_queries:
        print(f"Processing sub-query: {sub_query}")
        query_embedding = openai.Embedding.create(input=[sub_query], model="text-embedding-ada-002")['data'][0]['embedding']
        
        all_embeddings_data = collection.get(include=['metadatas', 'embeddings'])
        all_embeddings = [item for item in all_embeddings_data['embeddings']]
        all_metadatas = [meta['text'] for meta in all_embeddings_data['metadatas']]
        
        similarities = cosine_similarity([query_embedding], all_embeddings)[0]
        top_matches_indices = np.argsort(similarities)[-3:][::-1]
        top_matches_texts = [all_metadatas[i] for i in top_matches_indices]
        all_retrieved_texts.extend(top_matches_texts)

        time.sleep(1)  # Add a delay to avoid rate limits

    consolidated_context = truncate_text(" ".join(all_retrieved_texts))
    prompt = f"Based on the following content:\n{consolidated_context}\nAnswer the question: {user_query}"

    print("Consolidated Context:", consolidated_context[:500])  # Debugging
    print("Generated prompt:", prompt)
    return prompt



Response Generation

In [10]:
def generate_multi_query_response(user_query):
    # Detect language of the user's input
    detected_language = detect_language(user_query)
    print("Detected language:", detected_language)

    # Use multi-query processing to get aggregated context from ChromaDB
    prompt = multi_query_processing(user_query)
    print("Consolidated Context from ChromaDB:", prompt)

    # Construct enhanced prompt based on detected language
    enhanced_prompt = (
        f"{get_prompt(detected_language)}\n\n"
        f"Respond in the same language as {user_query}"
        f"Please answer strictly based on the following information retrieved from the database:\n{prompt}\n\n"
        f"Database Context:\n{prompt}\n\n"
        f"Answer as if you were directly recounting your experience as Princess Zelda, with the question: {user_query}\n"
        f"Translate your final answer into {detected_language}."
        "Answer in the format:\nAnswer: <your response here>"
    )

    # Generate a response using the consolidated context with streaming enabled
    response_chunks = []  # Store chunks for final response
    response_text = ""

    # Streaming API call
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": enhanced_prompt}],
        max_tokens=150,
        temperature=0.7,
        stream=True  # Enable streaming
    )

    # Process each streamed chunk as it arrives
    for chunk in response:
        # Each chunk has the format {'choices': [{'delta': {'content': 'text'}}]}
        chunk_content = chunk['choices'][0].get('delta', {}).get('content', "")
        response_text += chunk_content  # Append chunk to the full response
        print(chunk_content, end="")  # Print each chunk immediately for streaming effect

    print("\nGenerated Answer:", response_text)  # Print the final answer
    return response_text

print(generate_multi_query_response("Quien era Sonia?"))



Detected language: es
Processing sub-query: Quien era Sonia?
Processing sub-query: Detect the language of Quien era Sonia?
Processing sub-query: Background on Quien era Sonia?
Processing sub-query: Historical context of Quien era Sonia?
Processing sub-query: Role of Quien era Sonia? in Hyrule
Processing sub-query: Significance of Quien era Sonia?Respond in the same language as Quien era Sonia?
Consolidated Context: Breath of the Wild was a game about loneliness, an amnesiac Hero uncovering the truth of his Kingdom’s collapse, learning to survive and overcome the hostile wilderness left after the Great Calamity. Its direct sequel, Tears of the Kingdom, is instead a game about connections, about bringing Hyrule’s next generation together to fight against an ancient, yet familiar evil. Both games are set in the Kingdom of Hyrule - a vast land steeped in history and legend. The story of this Kingdom stretches 
Generated prompt: Based on the following content:
Breath of the Wild was a game 

In [11]:
print(generate_multi_query_response("Quien era Sonia?"))


Detected language: es
Processing sub-query: Quien era Sonia?
Processing sub-query: Detect the language of Quien era Sonia?
Processing sub-query: Background on Quien era Sonia?
Processing sub-query: Historical context of Quien era Sonia?
Processing sub-query: Role of Quien era Sonia? in Hyrule
Processing sub-query: Significance of Quien era Sonia?Respond in the same language as Quien era Sonia?
Consolidated Context: Breath of the Wild was a game about loneliness, an amnesiac Hero uncovering the truth of his Kingdom’s collapse, learning to survive and overcome the hostile wilderness left after the Great Calamity. Its direct sequel, Tears of the Kingdom, is instead a game about connections, about bringing Hyrule’s next generation together to fight against an ancient, yet familiar evil. Both games are set in the Kingdom of Hyrule - a vast land steeped in history and legend. The story of this Kingdom stretches 
Generated prompt: Based on the following content:
Breath of the Wild was a game 

In [12]:
print(generate_multi_query_response("Explica quem são os Zonai"))


Detected language: pt
Processing sub-query: Explica quem são os Zonai
Processing sub-query: Detect the language of Explica quem são os Zonai
Processing sub-query: Background on Explica quem são os Zonai
Processing sub-query: Historical context of Explica quem são os Zonai
Processing sub-query: Role of Explica quem são os Zonai in Hyrule
Processing sub-query: Significance of Explica quem são os ZonaiRespond in the same language as Explica quem são os Zonai
Consolidated Context: Breath of the Wild was a game about loneliness, an amnesiac Hero uncovering the truth of his Kingdom’s collapse, learning to survive and overcome the hostile wilderness left after the Great Calamity. Its direct sequel, Tears of the Kingdom, is instead a game about connections, about bringing Hyrule’s next generation together to fight against an ancient, yet familiar evil. Both games are set in the Kingdom of Hyrule - a vast land steeped in history and legend. The story of this Kingdom stretches 
Generated prompt:

In [13]:
#print(generate_multi_query_response("What are Zonai devices?"))


# Agent Configuration and Memory Setup:


Define Agent with LangChain

In [14]:
#Agent Config and Memory Setup:

# Initialize OpenAI model for the agent
llm = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=os.getenv("OPENAI_API_KEY"))

# Define memory to retain conversation continuity
memory = ConversationBufferMemory(input_key="input", output_key="output", return_messages=True)

# Define a tool that uses `generate_multi_query_response`
tools = [
    Tool(
        name="SearchChromaDB",
        func=generate_multi_query_response,  # Use the enhanced response generation function
        description="Detect the language of the user's input and answer in the same language based on relevant information retrieved from ChromaDB as Princess Zelda, staying fully in character."
    )
]

# Initialize the agent with memory and tools
agent = initialize_agent(
    tools=tools,
    llm=llm,
    agent_type=AgentType.CONVERSATIONAL_REACT_DESCRIPTION,
    memory=memory,
    verbose=True,
    handle_parsing_errors=True
)
agent.run({"input": "Quem és?"})   # Portuguese




> Entering new  chain...
Devo descobrir a língua em que a pergunta foi feita antes de responder.
Action: SearchChromaDB
Action Input: "Quem és?"Detected language: ca
Processing sub-query: Quem és?
Processing sub-query: Detect the language of Quem és?
Processing sub-query: Background on Quem és?
Processing sub-query: Historical context of Quem és?
Processing sub-query: Role of Quem és? in Hyrule
Processing sub-query: Significance of Quem és?Respond in the same language as Quem és?
Consolidated Context: Breath of the Wild was a game about loneliness, an amnesiac Hero uncovering the truth of his Kingdom’s collapse, learning to survive and overcome the hostile wilderness left after the Great Calamity. Its direct sequel, Tears of the Kingdom, is instead a game about connections, about bringing Hyrule’s next generation together to fight against an ancient, yet familiar evil. Both games are set in the Kingdom of Hyrule - a vast land steeped in history and legend. The story of this Kingdom s

'Sou a Princesa Zelda, descendente dos deuses Zonai e dos primeiros Hylians, transportada para o passado do Reino de Hyrule.'

# Voice Interaction

In [ ]:
#Voice Interaction

def handle_input(input_text=None, input_audio=None):
    # Check if audio is provided, prioritize it over text input
    if input_audio:
        # Transcribe audio
        transcription = whisper_model.transcribe(input_audio)["text"]
    elif input_text:
        transcription = input_text  # Use text input directly if no audio
    else:
        return "Please provide either text input or audio input."

    # Process the transcription with LangChain agent
    response = agent.run(transcription)
    return response

# Set up Gradio Interface
gr_interface = gr.Interface(
    fn=handle_input,
    inputs=[
        gr.Textbox(label="What brings you here, beloved Hyrulean?"),  # Text input
        gr.Audio(source="microphone", type="filepath", label="Or record your question")  # Audio input
    ],
    outputs="text",
    title="The Sheikah Slate",
    description="Ask me anything about the events of Tears of the Kingdom"
)

# Launch Gradio app
gr_interface.launch()

IMPORTANT: You are using gradio version 3.36.1, however version 4.44.1 is available, please upgrade.
--------
Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.




> Entering new  chain...
Debo buscar información sobre Rauru en ChromaDB para poder responder con precisión.
Action: SearchChromaDB
Action Input: RauruDetected language: lt
Processing sub-query: Rauru
Processing sub-query: Detect the language of Rauru
Processing sub-query: Background on Rauru
Processing sub-query: Historical context of Rauru
Processing sub-query: Role of Rauru in Hyrule
Processing sub-query: Significance of RauruRespond in the same language as Rauru
Consolidated Context: a different race entirely. The Ancient Hero’s Aspect, granted to Link for completing all of Rauru’s Shrines of Light, transforms Link into the Ancient Hero - the warrior shown on Impa’s tapestry. This Hero wears Zonaite Armour, which would have been incredibly ancient in this time, but he doesn’t seem to be a Zonai: his ears are much smaller, and his skin is more green than the Zonai’s grey. He also has red hair, and a tail - neither of which are present on Rauru or Mineru. Whatever the truth ab
Gene